In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from uniflow.transform.client import Client
from uniflow.transform.config import TransformOpenAIConfig
from uniflow.model.config import OpenAIModelConfig
from langchain.document_loaders import PyPDFLoader
from uniflow.schema import Context, GuidedPrompt

load_dotenv()

/opt/conda/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
from uniflow.op.transform.md_op import SplitMarkdownOp
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [4]:
markdown_file = "README.md"

In [5]:
dir_cur = os.getcwd()
input_file = os.path.join(f"{dir_cur}", markdown_file)
print(input_file)

/home/ubuntu/uniflow/example/transform/README.md


In [6]:
markdown_str = ""
with open(input_file, 'r') as file:
   markdown_str = file.read()

In [7]:
markdown_str

'# Examples\n## Base Config\nThe base `Config` is the base configuration that all other configurations inherit from. Here are the default parameters:\n\n| Parameter | Type | Default | Description |\n| --- | --- | --- | --- |\n| `flow_name` | `str` | [ModelFlow] | The name of the flow to run. |\n| `guided_prompt_template` | `GuidedPrompt` | [Default](../../README.md#2-prompting) | The template to use for the guided prompt. |\n| `num_threads` | `int` | 1 | The number of threads to use. |\n| `model_config` | `ModelConfig` | `ModelConfig` | The model configuration to use. |\n\nHere are the default parameters for the `ModelConfig`:\n\n| Parameter | Type | Default | Description |\n| --- | --- | --- | --- |\n| `model_name` | `str` | `gpt-3.5-turbo-1106` | The name of the model to use. |\n\nThe [model.ipynb notebook](./model.ipynb) shows a basic example of how to use the base `Config`, where it also passes the `OpenAIModelConfig` as a `model_config` argument.\n\n## OpenAIConfig\nThe `OpenAICon

In [8]:
s = SplitMarkdownOp().header_splitter(markdown_str)
s

[{'content': '## Base Config', 'metadata': {'Header 1': 'Examples'}},
 {'content': 'The base `Config` is the base configuration that all other configurations inherit from. Here are the default parameters:\nThe base `Config` is the base configuration that all other configurations inherit from. Here are the default parameters:\nThe base `Config` is the base configuration that all other configurations inherit from. Here are the default parameters:\nThe base `Config` is the base configuration that all other configurations inherit from. Here are the default parameters:\n| Parameter | Type | Default | Description |\n| Parameter | Type | Default | Description |\n| Parameter | Type | Default | Description |\n| Parameter | Type | Default | Description |\n| --- | --- | --- | --- |\n| --- | --- | --- | --- |\n| --- | --- | --- | --- |\n| --- | --- | --- | --- |\n| `flow_name` | `str` | [ModelFlow] | The name of the flow to run. |\n| `flow_name` | `str` | [ModelFlow] | The name of the flow to run.

In [9]:
headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_str)
md_header_splits

[Document(page_content='The base `Config` is the base configuration that all other configurations inherit from. Here are the default parameters:  \n| Parameter | Type | Default | Description |\n| --- | --- | --- | --- |\n| `flow_name` | `str` | [ModelFlow] | The name of the flow to run. |\n| `guided_prompt_template` | `GuidedPrompt` | [Default](../../README.md#2-prompting) | The template to use for the guided prompt. |\n| `num_threads` | `int` | 1 | The number of threads to use. |\n| `model_config` | `ModelConfig` | `ModelConfig` | The model configuration to use. |  \nHere are the default parameters for the `ModelConfig`:  \n| Parameter | Type | Default | Description |\n| --- | --- | --- | --- |\n| `model_name` | `str` | `gpt-3.5-turbo-1106` | The name of the model to use. |  \nThe [model.ipynb notebook](./model.ipynb) shows a basic example of how to use the base `Config`, where it also passes the `OpenAIModelConfig` as a `model_config` argument.', metadata={'Header 1': 'Examples', 'He

In [10]:
input_file = "../../" + markdown_file
markdown_str = ""
with open(input_file, 'r') as file:
   markdown_str = file.read()
markdown_str

'# 🌊 uniflow\n<p align="center">\n  <a href="/LICENSE"><img alt="License Apache-2.0" src="https://img.shields.io/github/license/cambioml/uniflow?style=flat-square"></a>\n  <a href="https://pypi.org/project/uniflow"><img src="https://img.shields.io/pypi/v/uniflow.svg" alt="pypi_status" /></a>\n  <a href="https://github.com/cambioml/uniflow/graphs/commit-activity"><img alt="Commit activity" src="https://img.shields.io/github/commit-activity/m/cambioml/uniflow?style=flat-square"/></a>\n  <a href="https://join.slack.com/t/cambiomlworkspace/shared_invite/zt-1zes33rmt-20Rag043uvExUaUdvt5_xQ"><img src="https://badgen.net/badge/Join/Community/cyan?icon=slack" alt="Slack" /></a>\n</p>\n\n`uniflow` is a unified interface to solve data augmentation problem for LLM training. It enables use of different LLMs, including [OpenAI](https://openai.com/product), [Huggingface](https://huggingface.co/mistralai/Mistral-7B-v0.1), and [LMQG](https://huggingface.co/lmqg) with a single interface. Using `uniflow

In [11]:
s = SplitMarkdownOp().header_splitter(markdown_str)
s

[{'content': '<p align="center">\n<p align="center">\n<p align="center">\n<p align="center">\n<a href="/LICENSE"><img alt="License Apache-2.0" src="https://img.shields.io/github/license/cambioml/uniflow?style=flat-square"></a>\n<a href="/LICENSE"><img alt="License Apache-2.0" src="https://img.shields.io/github/license/cambioml/uniflow?style=flat-square"></a>\n<a href="/LICENSE"><img alt="License Apache-2.0" src="https://img.shields.io/github/license/cambioml/uniflow?style=flat-square"></a>\n<a href="/LICENSE"><img alt="License Apache-2.0" src="https://img.shields.io/github/license/cambioml/uniflow?style=flat-square"></a>\n<a href="https://pypi.org/project/uniflow"><img src="https://img.shields.io/pypi/v/uniflow.svg" alt="pypi_status" /></a>\n<a href="https://pypi.org/project/uniflow"><img src="https://img.shields.io/pypi/v/uniflow.svg" alt="pypi_status" /></a>\n<a href="https://pypi.org/project/uniflow"><img src="https://img.shields.io/pypi/v/uniflow.svg" alt="pypi_status" /></a>\n<a h

In [12]:
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_str)
md_header_splits

[Document(page_content='<p align="center">\n<a href="/LICENSE"><img alt="License Apache-2.0" src="https://img.shields.io/github/license/cambioml/uniflow?style=flat-square"></a>\n<a href="https://pypi.org/project/uniflow"><img src="https://img.shields.io/pypi/v/uniflow.svg" alt="pypi_status" /></a>\n<a href="https://github.com/cambioml/uniflow/graphs/commit-activity"><img alt="Commit activity" src="https://img.shields.io/github/commit-activity/m/cambioml/uniflow?style=flat-square"/></a>\n<a href="https://join.slack.com/t/cambiomlworkspace/shared_invite/zt-1zes33rmt-20Rag043uvExUaUdvt5_xQ"><img src="https://badgen.net/badge/Join/Community/cyan?icon=slack" alt="Slack" /></a>\n</p>  \n`uniflow` is a unified interface to solve data augmentation problem for LLM training. It enables use of different LLMs, including [OpenAI](https://openai.com/product), [Huggingface](https://huggingface.co/mistralai/Mistral-7B-v0.1), and [LMQG](https://huggingface.co/lmqg) with a single interface. Using `unifl